In [ ]:
#pip install pca

In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
from pca import pca
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import FactorAnalysis

In [22]:
output = pd.DataFrame()
N = 2000

for beta1 in [0.1,1,10]:
    for beta2 in [0.1,1,10]:
        for covariance in [-0.9,-0.5,0,0.5,0.9]:
            for p in [5,10,20,50]:
                counter = 0
                if beta1 == 1:
                    counter += 1
                if beta2 == 1:
                    counter += 1
                if covariance == 0.5:
                    counter += 1
                if p == 5:
                    counter+=1
                if counter >= 3:
                    for k in range(1):
                        # Initialize Lists
                        pca_coef = []
                        mismeasured_coef = []
                        mismeasured_allvar_coef = []
                        mismeasured_avg_coef = []
                        iv_coef = []
                        true_val_coef =[]

                        # Create variables
                        vars_mean = [0,0,0]
                        vars_cov = np.array([[1,covariance,0],
                                             [covariance,1,0],
                                             [0,0,1]])
                        vars_ = pd.DataFrame(np.random.multivariate_normal(vars_mean, vars_cov, N), columns = ['x','true_z','u'])
                        vars_['y'] = beta1 * vars_['x'] + beta2 * vars_['true_z'] + vars_['u']

                        # Create measurement errors
                        errors_mean = np.zeros(p)
                        errors_cov = np.zeros((p,p))
                        for i in range(p):
                            for j in range(p):
                                if i == j:
                                    errors_cov[i,j] = 1

                        errors = np.random.multivariate_normal(errors_mean, errors_cov, N)
                        z_vars = []
                        for i in range(p):
                            z_vars.append('z'+str(i+1))
                        mismeasured_z = pd.DataFrame(errors, columns = z_vars)
                        for i in mismeasured_z.columns:
                            mismeasured_z[i] = mismeasured_z[i] + vars_['true_z']

                        # Do feature scaling (normalize to mean 0 and variance 1)
                        scaled_mismeasured_z = mismeasured_z.copy()
                        for i in mismeasured_z.columns:
                            scaled_mismeasured_z[i] = (mismeasured_z[i] - mismeasured_z[i].mean()) / mismeasured_z[i].std()

                        # Use PCA on the mismeasured values
                        pca_model = pca()
                        pca_results = pca_model.fit_transform(scaled_mismeasured_z)
                        pca_z = pca_results['PC']['PC1']

                        # Average mismeasured variables:
                        vars_['avg_mismeasured_z'] = mismeasured_z[z_vars].mean(axis=1)

                        # Add relevant variables to vars_ dataframe
                        vars_[mismeasured_z.columns] = mismeasured_z
                        vars_['pca_z'] = pca_z

                        # Mismeasured Results
                        model_mismeasured = sm.OLS(vars_['y'],vars_[['x','z1']])
                        results_mismeasured = model_mismeasured.fit()
                        mismeasured_coef.append(results_mismeasured.params[0])

                        # All Variables Mismeasured Results
                        tot_vars = ['x']
                        tot_vars.extend(z_vars)
                        model_mismeasured_allvar = sm.OLS(vars_['y'],vars_[tot_vars])
                        results_mismeasured_allvar = model_mismeasured_allvar.fit()
                        mismeasured_allvar_coef.append(results_mismeasured_allvar.params[0])

                        # Average Mismeasured Variables Results
                        model_mismeasured_avg = sm.OLS(vars_['y'],vars_[['x','avg_mismeasured_z']])
                        results_mismeasured_avg = model_mismeasured_avg.fit()
                        mismeasured_avg_coef.append(results_mismeasured_avg.params[0])

                        # PCA Results
                        model_pca = sm.OLS(vars_['y'],vars_[['x','pca_z']])
                        results_pca = model_pca.fit()
                        pca_coef.append(results_pca.params[0])

                        # Instrumental Variables Results
                        z_string = mismeasured_z.columns[1]
                        for z in mismeasured_z.columns[2:]:
                            z_string = z_string + ' + ' + z
                        # Create the predicted value of z1
                        vars_['pred_z1'] = smf.ols("z1 ~ " + z_string, data = mismeasured_z).fit().predict()
                        iv_results = smf.ols("y ~ x + pred_z1 -1", data = vars_).fit()
                        iv_coef.append(iv_results.params[0])
                        
                        # True Results
                        model_true = sm.OLS(vars_['y'],vars_[['x','true_z']])
                        results_true = model_true.fit()
                        true_val_coef.append(results_true.params[0])

                        # Output Findings
                        new_output = pd.DataFrame()
                        new_output['mismeasured_coef'] = mismeasured_coef
                        new_output['mismeasured_allvar_coef'] = mismeasured_allvar_coef
                        new_output['mismeasured_avg_coef'] = mismeasured_avg_coef
                        new_output['pca_coef'] = pca_coef
                        new_output['true_val_coef'] = true_val_coef
                        new_output['iv_coef'] = iv_coef
                        new_output['covariance'] = vars_cov[0][1]
                        new_output['beta1'] = beta1
                        new_output['beta2'] = beta2
                        new_output['p'] = p
                        output = output.append(new_output)
        
output

[pca] >Processing dataframe..
[pca] >The PCA reduction is performed to capture [95.0%] explained variance using the [5] columns of the input data.
[pca] >Fitting using PCA..
[pca] >Computing loadings and PCs..
[pca] >Computing explained variance..
[pca] >Number of components is [5] that covers the [95.00%] explained variance.
[pca] >Outlier detection using Hotelling T2 test with alpha=[0.05] and n_components=[5]
[pca] >Outlier detection using SPE/DmodX with n_std=[2]
[pca] >Processing dataframe..
[pca] >The PCA reduction is performed to capture [95.0%] explained variance using the [5] columns of the input data.
[pca] >Fitting using PCA..
[pca] >Computing loadings and PCs..
[pca] >Computing explained variance..
[pca] >Number of components is [5] that covers the [95.00%] explained variance.
[pca] >Outlier detection using Hotelling T2 test with alpha=[0.05] and n_components=[5]
[pca] >Outlier detection using SPE/DmodX with n_std=[2]
[pca] >Processing dataframe..
[pca] >The PCA reduction i

,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,covariance,beta1,beta2,p
0,0.377031,0.218752,0.217967,0.218737,0.123921,0.245400,0.5,0.1,1.0,5
0,1.006426,0.988041,0.987680,0.987556,0.975221,0.990657,0.5,1.0,0.1,5
0,0.235907,0.567432,0.568107,0.566743,1.071999,0.527989,-0.9,1.0,1.0,5
0,0.695738,0.905078,0.905190,0.903797,1.010369,0.884765,-0.5,1.0,1.0,5
0,0.981133,0.997965,0.997092,0.995853,0.985335,0.999879,0.0,1.0,1.0,5
0,1.280264,1.095359,1.096561,1.096687,1.004651,1.122497,0.5,1.0,1.0,5
0,1.296151,1.061114,1.063146,1.063970,0.992711,1.070037,0.5,1.0,1.0,10
0,1.274212,1.050924,1.046490,1.046366,1.012069,1.053971,0.5,1.0,1.0,20
0,1.330294,1.079765,1.073335,1.073230,1.064850,1.087304,0.5,1.0,1.0,50
0,1.733265,1.469905,1.470406,1.473297,1.012772,1.531346,0.9,1.0,1.0,5


In [23]:
output

,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,covariance,beta1,beta2,p
0,0.377031,0.218752,0.217967,0.218737,0.123921,0.245400,0.5,0.1,1.0,5
0,1.006426,0.988041,0.987680,0.987556,0.975221,0.990657,0.5,1.0,0.1,5
0,0.235907,0.567432,0.568107,0.566743,1.071999,0.527989,-0.9,1.0,1.0,5
0,0.695738,0.905078,0.905190,0.903797,1.010369,0.884765,-0.5,1.0,1.0,5
0,0.981133,0.997965,0.997092,0.995853,0.985335,0.999879,0.0,1.0,1.0,5
0,1.280264,1.095359,1.096561,1.096687,1.004651,1.122497,0.5,1.0,1.0,5
0,1.296151,1.061114,1.063146,1.063970,0.992711,1.070037,0.5,1.0,1.0,10
0,1.274212,1.050924,1.046490,1.046366,1.012069,1.053971,0.5,1.0,1.0,20
0,1.330294,1.079765,1.073335,1.073230,1.064850,1.087304,0.5,1.0,1.0,50
0,1.733265,1.469905,1.470406,1.473297,1.012772,1.531346,0.9,1.0,1.0,5


In [8]:
output.to_csv('C://Users//paulo//Documents//Econometrics and ML//Final Project//n_2000_results.csv')

In [28]:
pd.set_option('display.max_rows', 10)

In [9]:
output['pca_ape'] = (output['pca_coef'] - output['beta1']).abs()/output['beta1']
output['mismeasured_coef_ape'] = (output['mismeasured_coef'] - output['beta1']).abs()/output['beta1']
output['mismeasured_allvar_coef_ape'] = (output['mismeasured_allvar_coef'] - output['beta1']).abs()/output['beta1']
output['mismeasured_avg_coef_ape'] = (output['mismeasured_avg_coef'] - output['beta1']).abs()/output['beta1']

In [10]:
output.groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape
0,-0.9,1.0,1.0,5,0.244402,0.539647,0.539679,0.539015,1.001093,0.460985,0.755598,0.460353,0.460321
1,-0.5,1.0,1.0,5,0.714149,0.894030,0.893980,0.893722,0.999705,0.106278,0.285851,0.105970,0.106020
2,0.0,1.0,1.0,5,0.998864,0.999293,0.999322,0.999327,0.999421,0.019596,0.021828,0.019635,0.019612
3,0.5,0.1,1.0,5,0.385756,0.206114,0.206042,0.206258,0.100858,1.062582,2.857558,1.061140,1.060423
4,0.5,1.0,0.1,5,1.028871,1.010995,1.010988,1.011023,1.000367,0.021316,0.030990,0.021309,0.021293
5,0.5,1.0,1.0,5,1.284736,1.103504,1.103538,1.103772,0.998664,0.103772,0.284736,0.103504,0.103538
6,0.5,1.0,1.0,10,1.285694,1.058323,1.058351,1.058652,0.999323,0.058835,0.285694,0.058523,0.058553
7,0.5,1.0,1.0,20,1.283820,1.029584,1.029578,1.029873,0.998295,0.032897,0.283820,0.032671,0.032673
8,0.5,1.0,1.0,50,1.284698,1.011160,1.011050,1.011357,0.998213,0.022726,0.284698,0.022883,0.022609
9,0.5,1.0,10.0,5,3.863553,2.054715,2.054825,2.057738,0.999296,1.057738,2.863553,1.054715,1.054825


In [11]:
output.groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape
0,-0.9,1.0,1.0,5,0.031691,0.041496,0.041420,0.041432,0.050989,0.041432,0.031691,0.041496,0.041420
1,-0.5,1.0,1.0,5,0.028706,0.027214,0.027115,0.027131,0.026416,0.027131,0.028706,0.027214,0.027115
2,0.0,1.0,1.0,5,0.026979,0.024444,0.024389,0.024388,0.022925,0.014520,0.015882,0.014563,0.014500
3,0.5,0.1,1.0,5,0.029326,0.027558,0.027524,0.027523,0.026162,0.275232,0.293260,0.275582,0.275237
4,0.5,1.0,0.1,5,0.023741,0.024820,0.024815,0.024836,0.025131,0.016842,0.020898,0.016807,0.016817
5,0.5,1.0,1.0,5,0.028270,0.027473,0.027440,0.027478,0.026511,0.027478,0.028270,0.027473,0.027440
6,0.5,1.0,1.0,10,0.028983,0.027226,0.027152,0.027186,0.026378,0.026786,0.028983,0.026792,0.026714
7,0.5,1.0,1.0,20,0.029216,0.026468,0.026320,0.026353,0.025664,0.022461,0.029216,0.022542,0.022358
8,0.5,1.0,1.0,50,0.027874,0.026194,0.025895,0.025911,0.025711,0.016837,0.027874,0.016931,0.016766
9,0.5,1.0,10.0,5,0.165269,0.101216,0.100996,0.101600,0.025588,0.101600,0.165269,0.101216,0.100996


In [12]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['covariance'] == 0.5)].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape
0,0.5,1.0,1.0,5,1.284736,1.103504,1.103538,1.103772,0.998664,0.103772,0.284736,0.103504,0.103538
1,0.5,1.0,1.0,10,1.285694,1.058323,1.058351,1.058652,0.999323,0.058835,0.285694,0.058523,0.058553
2,0.5,1.0,1.0,20,1.283820,1.029584,1.029578,1.029873,0.998295,0.032897,0.283820,0.032671,0.032673
3,0.5,1.0,1.0,50,1.284698,1.011160,1.011050,1.011357,0.998213,0.022726,0.284698,0.022883,0.022609


In [16]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['covariance'] == 0.5)].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape
0,0.5,1.0,1.0,5,0.028270,0.027473,0.027440,0.027478,0.026511,0.027478,0.028270,0.027473,0.027440
1,0.5,1.0,1.0,10,0.028983,0.027226,0.027152,0.027186,0.026378,0.026786,0.028983,0.026792,0.026714
2,0.5,1.0,1.0,20,0.029216,0.026468,0.026320,0.026353,0.025664,0.022461,0.029216,0.022542,0.022358
3,0.5,1.0,1.0,50,0.027874,0.026194,0.025895,0.025911,0.025711,0.016837,0.027874,0.016931,0.016766


In [13]:
output.loc[(output['p']==5)&(output['beta2']==1)&(output['covariance'] == 0.5)].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape
0,0.5,0.1,1.0,5,0.385756,0.206114,0.206042,0.206258,0.100858,1.062582,2.857558,1.061140,1.060423
1,0.5,1.0,1.0,5,1.284736,1.103504,1.103538,1.103772,0.998664,0.103772,0.284736,0.103504,0.103538
2,0.5,10.0,1.0,5,10.286180,10.105493,10.105437,10.105679,9.999465,0.010568,0.028618,0.010549,0.010544
3,0.5,100.0,1.0,5,100.286184,100.105593,100.105580,100.105846,100.000166,0.001058,0.002862,0.001056,0.001056


In [17]:
output.loc[(output['p']==5)&(output['beta2']==1)&(output['covariance'] == 0.5)].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape
0,0.5,0.1,1.0,5,0.029326,0.027558,0.027524,0.027523,0.026162,0.275232,0.293260,0.275582,0.275237
1,0.5,1.0,1.0,5,0.028270,0.027473,0.027440,0.027478,0.026511,0.027478,0.028270,0.027473,0.027440
2,0.5,10.0,1.0,5,0.028685,0.026982,0.027032,0.027022,0.025972,0.002702,0.002869,0.002698,0.002703
3,0.5,100.0,1.0,5,0.028351,0.027634,0.027657,0.027696,0.026388,0.000277,0.000284,0.000276,0.000277


In [14]:
output.loc[(output['beta1']==1)&(output['p']==5)&(output['covariance'] == 0.5)].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape
0,0.5,1.0,0.1,5,1.028871,1.010995,1.010988,1.011023,1.000367,0.021316,0.030990,0.021309,0.021293
1,0.5,1.0,1.0,5,1.284736,1.103504,1.103538,1.103772,0.998664,0.103772,0.284736,0.103504,0.103538
2,0.5,1.0,10.0,5,3.863553,2.054715,2.054825,2.057738,0.999296,1.057738,2.863553,1.054715,1.054825
3,0.5,1.0,100.0,5,29.554625,11.519244,11.519468,11.546792,0.999807,10.546792,28.554625,10.519244,10.519468


In [18]:
output.loc[(output['beta1']==1)&(output['p']==5)&(output['covariance'] == 0.5)].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape
0,0.5,1.0,0.1,5,0.023741,0.024820,0.024815,0.024836,0.025131,0.016842,0.020898,0.016807,0.016817
1,0.5,1.0,1.0,5,0.028270,0.027473,0.027440,0.027478,0.026511,0.027478,0.028270,0.027473,0.027440
2,0.5,1.0,10.0,5,0.165269,0.101216,0.100996,0.101600,0.025588,0.101600,0.165269,0.101216,0.100996
3,0.5,1.0,100.0,5,1.516332,0.974031,0.971576,0.972906,0.025473,0.972906,1.516332,0.974031,0.971576


In [15]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['p'] == 5)].groupby(['covariance','beta1','beta2','p']).mean().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape
0,-0.9,1.0,1.0,5,0.244402,0.539647,0.539679,0.539015,1.001093,0.460985,0.755598,0.460353,0.460321
1,-0.5,1.0,1.0,5,0.714149,0.894030,0.893980,0.893722,0.999705,0.106278,0.285851,0.105970,0.106020
2,0.0,1.0,1.0,5,0.998864,0.999293,0.999322,0.999327,0.999421,0.019596,0.021828,0.019635,0.019612
3,0.5,1.0,1.0,5,1.284736,1.103504,1.103538,1.103772,0.998664,0.103772,0.284736,0.103504,0.103538
4,0.9,1.0,1.0,5,1.755877,1.462156,1.462116,1.462879,1.001033,0.462879,0.755877,0.462156,0.462116


In [19]:
output.loc[(output['beta1']==1)&(output['beta2']==1)&(output['p'] == 5)].groupby(['covariance','beta1','beta2','p']).std().reset_index()

,covariance,beta1,beta2,p,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape
0,-0.9,1.0,1.0,5,0.031691,0.041496,0.041420,0.041432,0.050989,0.041432,0.031691,0.041496,0.041420
1,-0.5,1.0,1.0,5,0.028706,0.027214,0.027115,0.027131,0.026416,0.027131,0.028706,0.027214,0.027115
2,0.0,1.0,1.0,5,0.026979,0.024444,0.024389,0.024388,0.022925,0.014520,0.015882,0.014563,0.014500
3,0.5,1.0,1.0,5,0.028270,0.027473,0.027440,0.027478,0.026511,0.027478,0.028270,0.027473,0.027440
4,0.9,1.0,1.0,5,0.031621,0.040444,0.040354,0.040268,0.050982,0.040268,0.031621,0.040444,0.040354
